In [ ]:
import requests
from bs4 import BeautifulSoup
from queue import Queue
import multiprocessing
import threading
import os, re, time
import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import WebDriverException
import configparser
from pymongo import MongoClient
import pymongo
import pandas as pd
import json
import signal
import stopit

In [ ]:
import concurrent
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# @stopit.threading_timeoutable('unexpected')
# def infinite():
#     while True:
#         pass
#     return 'whatever'
# infinite(timeout=5)

In [ ]:
### init and read config
config = configparser.ConfigParser()
config.read('config.ini')

MONGO_DB = config["Database"]["MONGO_DB"]
MONGO_HOST=config["Database"]["HOST"]
MONGO_USER = config["Database"]["MONGO_USER"]
MONGO_PASS = config["Database"]["MONGO_PASS"]

mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % (MONGO_USER, MONGO_PASS, MONGO_HOST, MONGO_DB)
conn = MongoClient(mongoURI)
db = conn[MONGO_DB]
handler = db['fanpage_post']

In [ ]:
def convertTime(stime):
    
    try:
        tmp = stime.split(" ")[1].split("|")[0]
        hour = re.sub("[^0-9]","",tmp.split(":")[0])
        minute = tmp.split(":")[1]
        if tmp[3:5] == "下午":
            hour = str(int(hour) + 12)
            if hour == "24":
                hour = "00"
        sdatetime = stime.split(" ")[0]+" "+hour+":"+minute
        postTime =  datetime.datetime.strptime(sdatetime, '%Y年%m月%d日 %H:%M')
    except:
        postTime = ""
        
    return postTime

In [ ]:
def commentParser(soup):
    
    commentList = []
    commenttable = soup.select(".commentable_item ul")[0].find_all("li",recursive=False) #recursive=False只找直接子結點

    for comment in commenttable:
        main = comment.find("div",{"aria-label":"留言"})
        name = main.find("img")['alt']
        try:
            content = main.find("span",{"dir":"ltr"}).get_text(strip=True)
        except:
            content = "" #只貼圖片，沒有文字內容
        try:
            stime = main.find("abbr")['data-tooltip-content']
        except:
            stime = main.find("abbr")['title']
        postTime = convertTime(stime)

        mainDic = {
            "name":name,
            "content":content,
            "postTime":postTime,
            #"stime":stime
        }
        subCommentList = []
        subComment = comment.find_all("div",{"aria-label":"留言回覆"})
        if subComment != None: #有子留言
            for sub in subComment:
                name = sub.find("img")['alt']
                try:
                    content = sub.find("span",{"dir":"ltr"}).get_text(strip=True)
                except:
                    content = "" #只貼圖片，沒有文字內容
                try:
                    stime = main.find("abbr")['data-tooltip-content']
                except:
                    stime = main.find("abbr")['title']
                postTime = convertTime(stime)

                subDic = {
                    "name":name,
                    "content":content,
                    "postTime":postTime,
                    #"stime":stime
                }
                subCommentList.append(subDic)

        mainDic["subComment"] = subCommentList
        commentList.append(mainDic)
    return commentList

In [ ]:
@stopit.threading_timeoutable("timeout")
def crawler(url):
    
    try:
        opts = Options()
        #opts.set_headless(headless=True)
        driver = webdriver.Firefox(options=opts)
        driver.get(url)
        time.sleep(5) # 等待稍後再說出現
        
        raw = ""
        comment = []
        error = False
        
        #點擊稍後在說        
        try:
            driver.find_element_by_link_text("稍後再說").click()
        except: #不用點稍後再說
            pass
        
        html_doc=driver.page_source
        soup = BeautifulSoup(html_doc, 'lxml')


        ## ver1
        #有可能右側欄出現同樣留言數的
        
        try:
            expand = soup.find("div", {"class": "userContentWrapper"}).find("form", {"class": "commentable_item"}).find("a", string=re.compile("^\s*[0-9]*\s*則留言"))
            num = len(soup.find_all("a",text = expand.text)) # X則留言
            element = driver.find_element_by_link_text(expand.text)
            driver.execute_script("arguments[0].scrollIntoView();", element) #滾輪滑倒X則留言位置
            driver.find_element_by_xpath("(//a[text()='"+expand.text+"'])[last()]").click()

            #driver.find_element_by_link_text(expand.text).click() #點擊X則留言
            
            ### test 
            
            #print(driver.find_element_by_xpath("//form[@class='commentable_item']").text)
            
            #print("//form[@class='commentable_item']//a[text()='"+expand.text+"']")
            #driver.find_element_by_xpath("//form[@class='commentable_item']//a[text()='"+expand.text+"']").click()
            #print("click",expand.text)
            #//*[@id="u_0_u"]/div/div[2]/div[1]/div/div[3]/span[1]/a
        except:
            error = "NotFindContent" #需要的登入的頁面 會找不到userContentWrapper


        ## ver2
        #找到展開留言element
        #expand = soup.find("div", {"class": "userContentWrapper"}).find("form", {"class": "commentable_item"}).find("a", string=re.compile("^\s*[0-9]*\s*則留言"))
        #一定要滑倒這個userContentWrapper裡X則留言的element，不然右側欄可能會有一樣X則留言
        #driver.find_element_by_css_selector(".commentable_item span[data-hover='tooltip'] a").click()
        #driver.execute_script("arguments[0].scrollIntoView();", element)
        #driver.find_element_by_link_text(expand.text).click()
        #element = driver.find_element_by_css_selector("div[class='userContentWrapper'] form[class='commentable_item']")
        #element = driver.find_element_by_xpath("//form[@class='commentable_item']/")

        # element = driver.find_element_by_link_text(expand)
        # driver.execute_script("arguments[0].scrollIntoView();", element)
        #點擊展開留言element

        #driver.find_element_by_css_selector("div[class='userContentWrapper'] form[class='commentable_ite'] a[data-ft='{'tn':'O'}']").click() #點擊"X則留言"  有可能發生右邊側欄其他貼文 留言數量一樣
        time.sleep(3)

#         try:
#             element = driver.find_element_by_link_text("最相關")
#             driver.execute_script("arguments[0].scrollIntoView();", element)
#             driver.find_element_by_link_text("最相關").click()
#             #print("click 最相關")
#             time.sleep(2)
#             html_doc=driver.page_source
#             soup = BeautifulSoup(html_doc, 'lxml')
#             driver.find_element_by_xpath("//ul[@role='menu']/li[3]").click() 
#         except:
#             #沒有「最相關」，而是在右邊顯示「最早」
#             pass
#         time.sleep(3)
#         html_doc=driver.page_source
#         soup = BeautifulSoup(html_doc, 'lxml')


        #展開所有留言，直到沒有
        while True:
            html_doc=driver.page_source
            soup = BeautifulSoup(html_doc, 'lxml')

            #展開留言有兩種text
            expand1 = soup.find("span", string=re.compile("^檢視另\s*[0-9]*\s*則留言"))
            expand2 = soup.find("span", string=re.compile("查看更多留言"))
            expand3 = soup.find("span", string=re.compile("^查看另\s*[0-9]*\s*則留言"))

            if expand1 != None: #沒有檢視另x則留言了，        
                element = driver.find_element_by_link_text(expand1.text)
                driver.execute_script("arguments[0].scrollIntoView();", element)
                driver.find_element_by_link_text(expand1.text).click()
                #print("展開留言")
                time.sleep(5)        
            elif expand2 != None:
                element = driver.find_element_by_link_text(expand2.text)
                driver.execute_script("arguments[0].scrollIntoView();", element)
                driver.find_element_by_link_text(expand2.text).click()
                #print("展開留言")
                time.sleep(5)
            elif expand3 != None:
                element = driver.find_element_by_link_text(expand3.text)
                driver.execute_script("arguments[0].scrollIntoView();", element)
                driver.find_element_by_link_text(expand3.text).click()
                #print("展開留言")
                time.sleep(5)
            else: #兩種都沒找到，沒有留言了
                #print("展開所有留言")
                break

        #print("展開所有留言")
        time.sleep(3)

        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #利用js滑到頁面最下方

        #展開所有子留言，直到沒有

        while True:
            html_doc=driver.page_source
            soup = BeautifulSoup(html_doc, 'lxml')
            expand1 = soup.find_all("span", string=re.compile("^檢視另\s*[0-9]*\s*則回覆"))
            expand2 = soup.find_all("span", string=re.compile("^\s*[0-9]*\s*則回覆"))
            expand3 = soup.find_all("a", string=re.compile("^查看更多"))

            if len(expand1) > 0: #沒有檢視另x則回覆了，跳出
                for button in expand1:
                    element = driver.find_element_by_link_text(button.text)
                    driver.execute_script("arguments[0].scrollIntoView();", element)
                    driver.find_element_by_link_text(button.text).click()
                    time.sleep(2)
                    #print("click",button.text)

            if len(expand2) > 0: #沒有檢視另x則回覆了，跳出
                for button in expand2:
                    element = driver.find_element_by_link_text(button.text)
                    driver.execute_script("arguments[0].scrollIntoView();", element)
                    driver.find_element_by_link_text(button.text).click()
                    time.sleep(2)
                    #print("click",button.text)

            if len(expand3) > 0: #沒有檢視另x則回覆了，跳出
                for button in expand3:
                    element = driver.find_element_by_link_text(button.text)
                    driver.execute_script("arguments[0].scrollIntoView();", element)
                    driver.find_element_by_link_text(button.text).click()
                    time.sleep(2)
                    #print("click",button.text)

            if len(expand1) == 0 and len(expand2) == 0 and len(expand3) == 0:
                #print("展開所有子留言")
                break


        #最終全部展開留言的html
        html_doc=driver.page_source
        soup = BeautifulSoup(html_doc, 'lxml')
        comment = commentParser(soup)
        error = False
        try:
            raw = soup.find("form", {"class": "commentable_item"})
        except: # 沒點擊到本文的展開留言
            error = "NotFindTable"
            raw = ""
            comment = []

    except WebDriverException:
        print("url error",url)
        error = "WebDriverException"

    except stopit.utils.TimeoutException:
        print("timeout exception")

        error = "TimeoutExceptino"
        
    except:
        error = "Other"
        
    dic = {
        "url":url,
        "comment_raw":str(raw),
        "comment":comment,
        "error":error
    }

    driver.close()
    
    return dic

In [ ]:
def thread_crawl(url,wait,q):
    result = crawler(url,timeout=wait)
    q.put(result)

In [ ]:
total = 0
max_bag = 5 #一次幾個子線程
count = 0
wait = 0 #主線程等待時間
tmp = []

#sort([("review_num", pymongo.ASCENDING)]) : 根據review_num做sort，review_num相近的頁面一起爬，這樣多線程比較省時間
#前制作業要先在資料庫建立review_num的ASCENDING index，否則會顯示RAM不夠
# with handler.find({'review_num':{'$gt':0},'comment_raw':{'$exists':False},'error':{'$exists':False}},{'url':1,'review_num':1},no_cursor_timeout=True).sort([("review_num", pymongo.ASCENDING)]) as cursor:
with handler.find({'review_num':{'$gt':0},'comment_raw':""},{'url':1,'review_num':1},no_cursor_timeout=True).sort([("review_num", pymongo.ASCENDING)]) as cursor:

    for row in cursor:
            
        if count == 0:
            start_time  = datetime.datetime.now()
            q = Queue()
            threads = []      
        #url = row['result link']
        wait = row['review_num']*30
        print("wait",wait)
        t = threading.Thread(target=thread_crawl, args=[row['url'],wait,q])
        #t.daemon = True
        threads.append(t)
        
        #wait = wait + row['review_num'] #根據review_num來決定總等待時間
        wait = []
        wait.append(row['review_num'])
        count = count + 1
        t.start()

        if count == max_bag: #每200筆清空一次thread
            for t in threads:
                t.join()
            #print(list(q.queue))
            for p in list(q.queue):
                handler.update_one(
                {'url':p['url']},
                {'$set':p}
                )
                
            total = total+max_bag
            time_elapsed = datetime.datetime.now() - start_time
            print(total,'Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
            count = 0
            wait = 0

    for t in threads:
        t.join()
    if len(list(q.queue)) > 0:
        #print(list(q.queue)[0])
        for p in list(q.queue):
            handler.update_one(
            {'url':p['url']},
            {'$set':p}
            )
    
    

    

In [ ]:
# total = 0
# max_bag = 5 #一次幾個子線程
# count = 0
# wait = 0 #主線程等待時間
# tmp = []

# #sort([("review_num", pymongo.ASCENDING)]) : 根據review_num做sort，review_num相近的頁面一起爬，這樣多線程比較省時間
# #前制作業要先在資料庫建立review_num的ASCENDING index，否則會顯示RAM不夠
# #with handler.find({'review_num':{'$gt':0},'comment_raw':{'$exists':False},'error':{'$exists':False}},{'url':1,'review_num':1},no_cursor_timeout=True).sort([("review_num", pymongo.ASCENDING)]) as cursor:
# with handler.find({'review_num':{'$gt':0},'comment_raw':""},{'url':1,'review_num':1},no_cursor_timeout=True).sort([("review_num", pymongo.ASCENDING)]) as cursor:

#     for row in cursor:
        
#         wait = row['review_num']*30
#         #print("wait",wait)
#         result = crawler(row['url'],wait)
#         handler.update_one(
#         {'url':result['url']},
#         {'$set':result}
#         )
#         count +=1
#         print(count)

        
    

    

## Test zone